# Widoki w Databricks

**Cel szkoleniowy:** Nauczenie się różnic między widokami a tabelami, zarządzania obiektami w Unity Catalog oraz tworzenia prostych pipeline'ów notebookowych w Databricks Jobs.

**Zakres tematyczny:**
- Różnice między VIEW, TABLE i DELTA TABLE
- Typy widoków: temp views, global temp views, persistent views
- Rejestracja obiektów w Unity Catalog
- Przeglądanie metadanych w Catalog Explorer
- Proste pipeline'y notebookowe
- Wprowadzenie do Databricks Jobs: taski, retry, harmonogramy

## Kontekst i wymagania

- **Dzień szkolenia**: Dzień 1 - Fundamentals & Exploration
- **Typ notebooka**: Demo
- **Wymagania techniczne**:
  - Databricks Runtime 13.0+ (zalecane: 14.3 LTS)
  - Unity Catalog włączony
  - Uprawnienia: CREATE TABLE, CREATE SCHEMA, SELECT, MODIFY
  - Klaster: Standard z minimum 2 workers

## Wstęp teoretyczny

### Czym są widoki i tabele?

W Databricks istnieją trzy główne typy obiektów przechowujących dane:

**1. TABLE (Tabela)**
- Fizyczny obiekt przechowujący dane na dysku
- Dane są zapisane w określonym formacie (Parquet, Delta, CSV, etc.)
- Wymaga miejsca na dysku
- Trwała (persistent) - przetrwa restart klastra

**2. DELTA TABLE (Tabela Delta)**
- Specjalny typ tabeli używający formatu Delta Lake
- Obsługuje ACID transactions
- Umożliwia Time Travel, MERGE, UPDATE, DELETE
- Zalecany format dla data lakehouse

**3. VIEW (Widok)**
- Wirtualny obiekt - nie przechowuje danych fizycznie
- Zapisuje tylko definicję zapytania SQL
- Wykonywany dynamicznie przy każdym odwołaniu
- Przydatny do enkapsulacji logiki biznesowej

### Typy widoków w Spark/Databricks:

| Typ widoku | Zasięg | Trwałość | Namespace |
|------------|--------|----------|----------|
| **Temp View** | Sesja Spark (notebook) | Do końca sesji | Lokalny w sesji |
| **Global Temp View** | Wszystkie sesje w klastrze | Do restartu klastra | `global_temp` database |
| **Persistent View** | Wszystkie użytkownicy | Trwały w Unity Catalog | Catalog.Schema |

### Kiedy używać którego?

- **Temp View**: Tymczasowe przetwarzanie w jednym notebooku
- **Global Temp View**: Współdzielenie danych między notebookami w tym samym klastrze
- **Persistent View**: Udostępnianie logiki biznesowej między zespołami
- **Delta Table**: Główny format do przechowywania danych w lakehouse

### Unity Catalog - hierarchia obiektów:

```
Metastore
  └── Catalog (np. kion_training)
      └── Schema/Database (np. bronze, silver, gold)
          ├── Tables
          ├── Views
          ├── Functions
          └── Volumes
```

## Inicjalizacja środowiska

Uruchom skrypt inicjalizacyjny dla per-user izolacji katalogów i schematów:

In [0]:
%run ../00_setup

## Konfiguracja

Import bibliotek i ustawienie zmiennych środowiskowych:

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from datetime import datetime

# Ścieżki do danych
CUSTOMERS_CSV = f"{DATASET_BASE_PATH}/customers/customers.csv"
ORDERS_JSON = f"{DATASET_BASE_PATH}/orders/orders_batch.json"
PRODUCTS_PARQUET = f"{DATASET_BASE_PATH}/products/products.parquet"

# Wyświetl kontekst użytkownika (zmienne z 00_setup)
print("=== Kontekst użytkownika ===")
print(f"Katalog: {CATALOG}")
print(f"Schema Bronze: {BRONZE_SCHEMA}")
print(f"Schema Silver: {SILVER_SCHEMA}")
print(f"Schema Gold: {GOLD_SCHEMA}")
print(f"Użytkownik: {raw_user}")

# Ustaw katalog i schemat jako domyślne
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {BRONZE_SCHEMA}")

print(f"\n Domyślny katalog: {CATALOG}")
print(f" Domyślny schemat: {BRONZE_SCHEMA}")

---

## Część 1: Przygotowanie danych testowych

### 1.1. Wczytanie danych do DataFrame

In [0]:
# Wczytaj dane klientów z CSV
customers_df = (
    spark.read
    .format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(CUSTOMERS_CSV)
)

print(f" Wczytano {customers_df.count()} klientów")
customers_df.display(5)

In [0]:
# Wczytaj dane zamówień z JSON
orders_df = (
    spark.read
    .format("json")
    .option("inferSchema", "true")
    .load(ORDERS_JSON)
)

print(f" Wczytano {orders_df.count()} zamówień")
orders_df.display(5)

---

## Część 2: Temporary Views (Widoki tymczasowe)

### 2.1. Tworzenie Temp View

**Temp View** istnieje tylko w bieżącej sesji Spark (notebooku).

In [0]:
# Utwórz temporary view z DataFrame
customers_df.createOrReplaceTempView("customers_temp_view")

print(" Utworzono temp view: customers_temp_view")

# Sprawdź, czy widok istnieje
print("\n Lista temporary views:")
spark.sql("SHOW VIEWS").filter("isTemporary = true").display(truncate=False)

### 2.2. Zapytanie do Temp View

In [0]:
# Zapytanie SQL do temp view
result_df = spark.sql("""
    SELECT country, COUNT(*) as customer_count
    FROM customers_temp_view
    GROUP BY country
    ORDER BY customer_count DESC
""")

print(" Liczba klientów według krajów:")
result_df.display()

### 2.3. Usunięcie Temp View

In [0]:
# Usuń temporary view
spark.catalog.dropTempView("customers_temp_view")

print(" Usunięto temp view: customers_temp_view")

# Weryfikacja
try:
    spark.sql("SELECT * FROM customers_temp_view LIMIT 1")
except Exception as e:
    print(f"[EXPECTED ERROR] Widok nie istnieje: {str(e)[:100]}")

---

## Część 3: Global Temporary Views

### 3.1. Tworzenie Global Temp View

**Global Temp View** jest dostępny dla wszystkich sesji w klastrze, ale tylko do restartu klastra.

In [0]:
# Utwórz global temporary view
orders_df.createOrReplaceGlobalTempView("orders_global_temp_view")

print(" Utworzono global temp view: orders_global_temp_view")
print(" Dostęp przez namespace: global_temp.orders_global_temp_view")

### 3.2. Zapytanie do Global Temp View

**Uwaga:** Musisz użyć prefiksu `global_temp`

In [0]:
# Zapytanie SQL do global temp view (z prefiksem global_temp)
result_df = spark.sql("""
    SELECT store_id, COUNT(*) as order_count, SUM(total_amount) as total_revenue
    FROM global_temp.orders_global_temp_view
    GROUP BY store_id
    ORDER BY total_revenue DESC
""")

print(" Statystyki zamówień według store_id:")
result_df.display()

### 3.3. Usunięcie Global Temp View

In [0]:
# Usuń global temporary view
spark.catalog.dropGlobalTempView("orders_global_temp_view")

print(" Usunięto global temp view: orders_global_temp_view")

---

## Część 4: Delta Tables (Tabele trwałe)

### 4.1. Tworzenie Delta Table w Unity Catalog

**Delta Table** to fizyczna tabela zapisana w formacie Delta Lake.

In [0]:
# Nazwa tabeli w Unity Catalog
CUSTOMERS_TABLE = f"{CATALOG}.{BRONZE_SCHEMA}.customers"

# Zapisz DataFrame jako Delta Table
customers_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(CUSTOMERS_TABLE)

print(f" Utworzono Delta Table: {CUSTOMERS_TABLE}")

# Sprawdź liczbę rekordów
count = spark.table(CUSTOMERS_TABLE).count()
print(f" Liczba rekordów w tabeli: {count}")

### 4.2. Metadane Delta Table

In [0]:
# Sprawdź szczegóły tabeli
print(" Szczegóły Delta Table:")
spark.sql(f"DESCRIBE DETAIL {CUSTOMERS_TABLE}").display(truncate=False)

# Sprawdź schemat tabeli
print("\n Schemat tabeli:")
spark.sql(f"DESCRIBE {CUSTOMERS_TABLE}").display(truncate=False)

### 4.3. Tworzenie drugiej tabeli (Orders)

In [0]:
# Nazwa tabeli zamówień
ORDERS_TABLE = f"{CATALOG}.{BRONZE_SCHEMA}.orders"

# Zapisz jako Delta Table
orders_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(ORDERS_TABLE)

print(f" Utworzono Delta Table: {ORDERS_TABLE}")
print(f" Liczba rekordów: {spark.table(ORDERS_TABLE).count()}")

---

## Część 5: Persistent Views (Widoki trwałe)

### 5.1. Tworzenie Persistent View w Unity Catalog

**Persistent View** jest zapisywany w Unity Catalog i dostępny dla wszystkich użytkowników.

In [0]:
# Utwórz persistent view z logiką biznesową
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {SILVER_SCHEMA}")
VIEW_NAME = f"customer_order_summary"

spark.sql(f"""
    CREATE OR REPLACE VIEW `{VIEW_NAME}` AS
    SELECT 
        c.customer_id,
        concat_ws(' ', c.first_name, c.last_name) as customer_name,
        c.country,
        COUNT(o.order_id) as total_orders,
        COALESCE(SUM(o.total_amount), 0) as total_spent,
        COALESCE(AVG(o.total_amount), 0) as avg_order_value
    FROM {CUSTOMERS_TABLE} c
    LEFT JOIN {ORDERS_TABLE} o ON c.customer_id = o.customer_id
    GROUP BY c.customer_id, c.first_name, c.last_name, c.country
""")

print(f" Utworzono persistent view: {VIEW_NAME}")

### 5.2. Zapytanie do Persistent View

In [0]:
# Zapytanie do widoku - top 10 klientów według wydatków
result_df = spark.sql(f"""
    SELECT 
        customer_name,
        country,
        total_orders,
        ROUND(total_spent, 2) as total_spent,
        ROUND(avg_order_value, 2) as avg_order_value
    FROM {VIEW_NAME}
    WHERE total_orders > 0
    ORDER BY total_spent DESC
    LIMIT 10
""")

print(" Top 10 klientów według wydatków:")
result_df.display(truncate=False)

### 5.3. Metadane Persistent View

In [0]:
# Sprawdź definicję widoku
print(" Definicja widoku:")
spark.sql(f"SHOW CREATE TABLE {VIEW_NAME}").display(truncate=False)

# Lista wszystkich widoków w schemacie
print(f"\n Widoki w schemacie {SILVER_SCHEMA}:")
spark.sql(f"SHOW VIEWS IN {CATALOG}.{SILVER_SCHEMA}").display(truncate=False)

---

## Część 6: Przeglądanie obiektów w Unity Catalog

### 6.1. Lista katalogów

In [0]:
# Wyświetl wszystkie dostępne katalogi
print(" Dostępne katalogi:")
spark.sql("SHOW CATALOGS").display(truncate=False)

### 6.2. Lista schematów w katalogu

In [0]:
# Wyświetl schematy w bieżącym katalogu
print(f" Schematy w katalogu {CATALOG}:")
spark.sql(f"SHOW SCHEMAS IN {CATALOG}").display(truncate=False)

### 6.3. Lista tabel w schemacie

In [0]:
# Wyświetl tabele w schemacie bronze
print(f" Tabele w schemacie {BRONZE_SCHEMA}:")
spark.sql(f"SHOW TABLES IN {CATALOG}.{BRONZE_SCHEMA}").display(truncate=False)

# Wyświetl widoki w schemacie silver
print(f"\n Widoki w schemacie {SILVER_SCHEMA}:")
spark.sql(f"SHOW VIEWS IN {CATALOG}.{SILVER_SCHEMA}").display(truncate=False)

### 6.4. Szczegółowe informacje o tabeli

In [0]:
# Sprawdź właściciela, lokalizację, format tabeli
print(f" Szczegóły tabeli {CUSTOMERS_TABLE}:")
spark.sql(f"DESCRIBE EXTENDED {CUSTOMERS_TABLE}").display(100, truncate=False)

---

## Część 7: Porównanie VIEW vs TABLE

### 7.1. Test wydajności: View vs Materialized Table

In [0]:
import time

# Test 1: Zapytanie do VIEW (obliczane on-the-fly)
start_view = time.time()
result_view = spark.sql(f"SELECT COUNT(*) as cnt FROM {VIEW_NAME}").collect()
time_view = time.time() - start_view

print(f"[BENCHMARK] Czas wykonania VIEW: {time_view:.3f} sekund")
print(f"[BENCHMARK] Wynik: {result_view[0]['cnt']} rekordów")

# Test 2: Materializacja wyniku jako tabela
MATERIALIZED_TABLE = f"{CATALOG}.{SILVER_SCHEMA}.customer_order_summary_materialized"

spark.sql(f"""
    CREATE OR REPLACE TABLE {MATERIALIZED_TABLE} AS
    SELECT * FROM {VIEW_NAME}
""")

# Test wydajności tabeli zmaterializowanej
start_table = time.time()
result_table = spark.sql(f"SELECT COUNT(*) as cnt FROM {MATERIALIZED_TABLE}").collect()
time_table = time.time() - start_table

print(f"\n[BENCHMARK] Czas wykonania MATERIALIZED TABLE: {time_table:.3f} sekund")
print(f"[BENCHMARK] Wynik: {result_table[0]['cnt']} rekordów")

# Porównanie
if time_view > time_table:
    speedup = (time_view - time_table) / time_view * 100
    print(f"\n[WYNIK] Materialized table była szybsza o {speedup:.1f}%")
else:
    print(f"\n[WYNIK] Różnica w wydajności nieznaczna dla tego datasetu")

### 7.2. Kiedy używać VIEW, a kiedy TABLE?

**Użyj VIEW gdy:**
- Chcesz enkapsulować logikę biznesową
- Dane źródłowe często się zmieniają
- Zapytanie jest proste i szybkie
- Nie chcesz duplikować danych

**Użyj MATERIALIZED TABLE gdy:**
- Zapytanie jest skomplikowane i wolne
- Dane są czytane bardzo często
- Dane źródłowe rzadko się zmieniają
- Potrzebujesz indeksowania (ZORDER) dla wydajności

---

## Część 8: Wprowadzenie do Databricks Jobs

### 8.1. Czym jest Databricks Job?

**Databricks Job** to mechanizm automatyzacji i orkiestracji notebooków, skryptów lub zapytań SQL.

**Kluczowe pojęcia:**
- **Job** - kontener dla jednego lub więcej tasków
- **Task** - pojedyncza jednostka pracy (notebook, skrypt, SQL, JAR)
- **Cluster** - zasoby obliczeniowe do wykonania zadania
- **Trigger** - sposób uruchomienia (manual, scheduled, continuous)
- **Retry** - automatyczne ponawianie w przypadku błędu

### 8.2. Typy tasków:

| Typ tasku | Opis | Use case |
|-----------|------|----------|
| **Notebook** | Wykonanie notebooka Databricks | ETL pipelines, data processing |
| **Python script** | Uruchomienie skryptu .py | Data validation, custom logic |
| **JAR** | Uruchomienie aplikacji Spark | Legacy Spark applications |
| **SQL** | Wykonanie zapytania SQL | Data transformation, reporting |
| **dbt** | Uruchomienie projektu dbt | Modern data transformation |
| **Delta Live Tables** | Pipeline DLT | Declarative ETL pipelines |

### 8.3. Harmonogramy (Schedules):

- **Manual** - uruchomienie na żądanie
- **Cron expression** - np. `0 0 * * *` (codziennie o północy)
- **Continuous** - ciągłe przetwarzanie (streaming)

### 8.4. Retry i timeout:

- **Max retries** - maksymalna liczba prób (0-3)
- **Retry interval** - czas między próbami (sekundy)
- **Timeout** - maksymalny czas wykonania tasku

### 8.5. Przykład prostego pipeline'u notebookowego

W praktyce pipeline składałby się z kilku notebooków wykonywanych sekwencyjnie:

```
Pipeline: Daily Customer Analytics
├── Task 1: Ingest raw data (notebook: 01_ingest.ipynb)
│   └── Retry: 2x, Timeout: 10 min
├── Task 2: Clean and transform (notebook: 02_transform.ipynb)
│   └── Depends on: Task 1
├── Task 3: Aggregate metrics (notebook: 03_aggregate.ipynb)
│   └── Depends on: Task 2
└── Task 4: Send notification (Python script: notify.py)
    └── Depends on: Task 3
```

**W Databricks UI:**
1. Workflows → Create Job
2. Add tasks (notebooks)
3. Skonfiguruj dependencies między taskami
4. Ustaw harmonogram (schedule)
5. Skonfiguruj cluster
6. Uruchom job

## Tworzenie prostego Job’a w UI – krok po kroku

1. Przejdź do **Workflows → Jobs** i kliknij **Create Job**.
2. Nazwij Job, wybierz typ taska **Notebook** i wskaż notebook (np. pipeline Bronze→Silver→Gold).
3. Wybierz cluster:
   - istniejący all-purpose (dev), albo
   - nowy job cluster z odpowiednim Databricks Runtime.
4. (Opcjonalnie) dodaj **parametry**:
   - job parameters,
   - lub mapowanie na widgety w notebooku (`dbutils.widgets.get`).
5. Skonfiguruj **schedule** (np. codziennie o 07:00) lub pozostaw uruchamianie ręczne.
6. Ustaw **Retry** (np. 2–3 próby) oraz ewentualny **timeout**.

Po pierwszych uruchomieniach:

- obserwuj czas wykonania,
- zaglądaj do Spark UI,
- sprawdzaj liczbę rekordów między warstwami (Bronze/Silver/Gold).

## Gdzie szukać logów i szczegółów wykonań Jobów?

Po uruchomieniu Databricks Jobów najważniejsze miejsca diagnostyczne:

1. **Job run details (Jobs → konkretny run)**:
   - status poszczególnych tasków,
   - czas wykonania,
   - liczba retry,
   - powiązany cluster.

2. **Spark UI**:
   - zakładka **SQL** – plany wykonania zapytań,
   - zakładka **Jobs** – DAG zadań Spark,
   - zakładka **Storage** – cache / shuffle,
   - zakładka **Environment** – konfiguracja clustra i aplikacji.

3. **Driver / Executor logs**:
   - szczegółowe stacktrace’y wyjątków Pythona/Scali,
   - logi aplikacyjne.

Dobre praktyki:

- Przy krytycznych pipeline’ach ETL zawsze sprawdzaj Spark UI przynajmniej przy pierwszych wdrożeniach.
- Jeśli Job kończy się statusem `SUCCESS`, ale dane wyglądają podejrzanie – porównaj liczbę wierszy i proste agregaty między warstwami.

### 8.6. Przekazywanie parametrów między notebookami

**Metoda 1: Widgets (dla pojedynczego notebooka)**

In [0]:
# Przykład: Tworzenie widgetu do parametryzacji
# (w Databricks to utworzy interaktywny element UI)

# dbutils.widgets.text("processing_date", "2024-01-01", "Processing Date")
# processing_date = dbutils.widgets.get("processing_date")

# W tym notebooku używamy bezpośrednio wartości
processing_date = "2024-01-01"
print(f" Processing date: {processing_date}")

**Metoda 2: dbutils.notebook.run (dla pipeline'u notebooków)**

```python
# Uruchomienie kolejnego notebooka z parametrami
result = dbutils.notebook.run(
    "./02_next_notebook",
    timeout_seconds=600,
    arguments={"date": "2024-01-01", "mode": "full"}
)
print(f"Notebook result: {result}")
```

---

## Część 9: Demonstracja - Prosty pipeline w jednym notebooku

### 9.1. Pipeline: Bronze → Silver → Gold

In [0]:
print("=== PIPELINE START ===")
print(f"Timestamp: {datetime.now()}")

# Step 1: Bronze - dane już wczytane wcześniej
print("\n[STEP 1: BRONZE] Dane surowe już istnieją")
print(f"  - Tabela customers: {spark.table(CUSTOMERS_TABLE).count()} rekordów")
print(f"  - Tabela orders: {spark.table(ORDERS_TABLE).count()} rekordów")

# Step 2: Silver - widok z logiką biznesową już istnieje
print("\n[STEP 2: SILVER] Widok customer_order_summary już istnieje")
silver_count = spark.sql(f"SELECT COUNT(*) as cnt FROM {VIEW_NAME}").collect()[0]['cnt']
print(f"  - Liczba rekordów w widoku: {silver_count}")

# Step 3: Gold - agregacja na poziomie kraju
print("\n[STEP 3: GOLD] Tworzenie agregacji na poziomie kraju")
GOLD_TABLE = f"{CATALOG}.{GOLD_SCHEMA}.country_metrics"

gold_df = spark.sql(f"""
    SELECT 
        country,
        COUNT(DISTINCT customer_id) as total_customers,
        SUM(total_orders) as total_orders,
        ROUND(SUM(total_spent), 2) as total_revenue,
        ROUND(AVG(avg_order_value), 2) as avg_order_value,
        current_timestamp() as updated_at
    FROM {VIEW_NAME}
    WHERE total_orders > 0
    GROUP BY country
    ORDER BY total_revenue DESC
""")

# Zapisz do Gold layer
gold_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(GOLD_TABLE)

print(f"  - Utworzono tabelę Gold: {GOLD_TABLE}")
print(f"  - Liczba krajów: {gold_df.count()}")

print("\n=== PIPELINE COMPLETED ===")
print(f"Timestamp: {datetime.now()}")

### 9.2. Weryfikacja wyniku pipeline'u

In [0]:
# Wyświetl wynik agregacji Gold
print(" Metryki według krajów (Gold Layer):")
spark.table(GOLD_TABLE).display(truncate=False)

# Sprawdź timestamp aktualizacji
latest_update = spark.sql(f"SELECT MAX(updated_at) as last_update FROM {GOLD_TABLE}").collect()[0]['last_update']
print(f"\n Ostatnia aktualizacja: {latest_update}")

---

## Walidacja i weryfikacja

### Checklist - Co powinieneś uzyskać:
- [x] Tabela `customers` w schemacie Bronze
- [x] Tabela `orders` w schemacie Bronze
- [x] Widok `customer_order_summary` w schemacie Silver
- [x] Tabela `country_metrics` w schemacie Gold
- [x] Zrozumienie różnic między VIEW, TEMP VIEW, GLOBAL TEMP VIEW
- [x] Znajomość podstaw Databricks Jobs

### Komendy weryfikacyjne:

In [0]:
print("=== WERYFIKACJA WYNIKÓW ===")

# 1. Sprawdź tabele Bronze
print("\n[1] Tabele Bronze:")
spark.sql(f"SHOW TABLES IN {CATALOG}.{BRONZE_SCHEMA}").display(truncate=False)

# 2. Sprawdź widoki Silver
print("\n[2] Widoki Silver:")
spark.sql(f"SHOW VIEWS IN {CATALOG}.{SILVER_SCHEMA}").display(truncate=False)

# 3. Sprawdź tabele Gold
print("\n[3] Tabele Gold:")
spark.sql(f"SHOW TABLES IN {CATALOG}.{GOLD_SCHEMA}").display(truncate=False)

# 4. Sprawdź liczby rekordów
print("\n[4] Liczba rekordów:")
print(f"  - customers (Bronze): {spark.table(CUSTOMERS_TABLE).count()}")
print(f"  - orders (Bronze): {spark.table(ORDERS_TABLE).count()}")
print(f"  - customer_order_summary (Silver VIEW): {spark.sql(f'SELECT COUNT(*) as cnt FROM {VIEW_NAME}').collect()[0]['cnt']}")
print(f"  - country_metrics (Gold): {spark.table(GOLD_TABLE).count()}")

print("\n[SUCCESS] Wszystkie obiekty zostały utworzone poprawnie!")

---

## Troubleshooting

### Problem 1: "Table or view not found"
**Objawy:**
- Błąd podczas zapytania SQL: `Table or view 'xyz' not found`

**Rozwiązanie:**
```python
# Sprawdź, czy obiekt istnieje
spark.sql("SHOW TABLES IN catalog.schema").display()

# Użyj pełnej ścieżki (three-level namespace)
spark.sql("SELECT * FROM catalog.schema.table_name")
```

### Problem 2: Temp view nie jest widoczny
**Objawy:**
- Temp view utworzony w jednym notebooku nie działa w innym

**Rozwiązanie:**
- Temp views są lokalne dla sesji - użyj Global Temp View lub Persistent View
- Global Temp View wymaga prefiksu `global_temp.`

### Problem 3: "Permission denied" przy tworzeniu tabeli
**Objawy:**
- Brak uprawnień do utworzenia tabeli w schemacie

**Rozwiązanie:**
```sql
-- Administrator musi nadać uprawnienia
GRANT CREATE TABLE ON SCHEMA catalog.schema TO `user@domain.com`;
GRANT MODIFY ON SCHEMA catalog.schema TO `user@domain.com`;
```

### Problem 4: View pokazuje stare dane
**Objawy:**
- Dane w widoku nie są aktualne mimo aktualizacji tabeli źródłowej

**Rozwiązanie:**
- Widoki są dynamiczne - powinny pokazywać aktualne dane
- Sprawdź cache: `spark.catalog.clearCache()`
- Dla danych historycznych użyj materializowanej tabeli

### Debugging tips:
- Sprawdź current catalog: `SELECT current_catalog()`
- Sprawdź current schema: `SELECT current_schema()`
- Lista wszystkich obiektów: `SHOW TABLES` i `SHOW VIEWS`
- Definicja widoku: `SHOW CREATE TABLE view_name`

---

## Best Practices

### Zarządzanie widokami:
- Używaj **Persistent Views** dla logiki biznesowej współdzielonej między zespołami
- Używaj **Temp Views** dla tymczasowych przekształceń w ramach jednego notebooka
- Dokumentuj logikę widoków w komentarzach SQL
- Nazwy widoków powinny jasno wskazywać ich przeznaczenie (np. `vw_customer_360`, `vw_daily_sales`)

### Zarządzanie tabelami:
- Zawsze używaj **Delta format** dla tabel w lakehouse
- Stosuj konwencję nazewnictwa: `{layer}_{domain}_{entity}` (np. `bronze_sales_transactions`)
- Dodawaj kolumny audytowe: `created_at`, `updated_at`, `source_system`
- Regularnie uruchamiaj `OPTIMIZE` i `VACUUM` dla tabel Delta

### Unity Catalog:
- Organizuj obiekty w logiczne schematy: bronze, silver, gold
- Używaj three-level namespace: `catalog.schema.table`
- Nadawaj uprawnienia na poziomie schematu, nie tabeli
- Regularnie przeglądaj logi audytowe

### Databricks Jobs:
- Dziel duże notebooki na mniejsze, modularne taski
- Używaj parametryzacji przez widgets
- Konfiguruj retry dla tasków podatnych na błędy (np. API calls)
- Monitoruj czas wykonania i ustawiaj sensowne timeouty
- Używaj alertów e-mail/Slack dla krytycznych pipeline'ów
- Testuj pipeline lokalnie przed wdrożeniem do produkcji

### Wydajność:
- Materializuj widoki, które są często używane w złożonych zapytaniach
- Używaj partycjonowania dla dużych tabel
- Cache'uj wyniki pośrednie w pipeline'ach
- Monitoruj Spark UI dla identyfikacji bottlenecks

---

## Podsumowanie

### Co zostało osiągnięte:
- Poznanie różnic między VIEW, TABLE i DELTA TABLE
- Tworzenie Temp Views, Global Temp Views i Persistent Views
- Rejestracja tabel i widoków w Unity Catalog
- Przeglądanie metadanych w Catalog Explorer
- Zrozumienie podstaw Databricks Jobs i pipeline'ów
- Implementacja prostego pipeline'u Bronze → Silver → Gold

### Kluczowe wnioski:
1. **VIEW vs TABLE**: View nie przechowuje danych, TABLE materializuje wyniki na dysku
2. **Delta Lake**: Zalecany format dla wszystkich tabel w lakehouse (ACID, Time Travel, MERGE)
3. **Unity Catalog**: Centralne zarządzanie metadanymi, three-level namespace
4. **Databricks Jobs**: Automatyzacja i orkiestracja notebooków w pipeline'y
5. **Medallion Architecture**: Bronze (raw) → Silver (cleaned) → Gold (aggregated)

### Quick Reference - Najważniejsze komendy:

| Operacja | PySpark | SQL |
|----------|---------|-----|
| Temp View | `df.createOrReplaceTempView("name")` | `CREATE TEMP VIEW name AS SELECT ...` |
| Global Temp View | `df.createOrReplaceGlobalTempView("name")` | `CREATE GLOBAL TEMP VIEW name AS SELECT ...` |
| Persistent View | - | `CREATE VIEW catalog.schema.name AS SELECT ...` |
| Delta Table | `df.write.format("delta").saveAsTable("name")` | `CREATE TABLE name USING DELTA AS SELECT ...` |
| Lista tabel | `spark.catalog.listTables()` | `SHOW TABLES IN catalog.schema` |
| Lista widoków | - | `SHOW VIEWS IN catalog.schema` |
| Metadane | - | `DESCRIBE EXTENDED table_name` |
| Drop view | `spark.catalog.dropTempView("name")` | `DROP VIEW IF EXISTS name` |

### Następne kroki:
- **Kolejny notebook**: `DZIEN_2_Lakehouse_Delta_Lake/01_delta_lake_operations.ipynb`
- **Warsztat praktyczny**: `workshops/03_views_basic_jobs_workshop.ipynb`
- **Materiały dodatkowe**: 
  - [Unity Catalog documentation](https://docs.databricks.com/data-governance/unity-catalog/index.html)
  - [Databricks Jobs documentation](https://docs.databricks.com/workflows/jobs/jobs.html)
  - [Delta Lake best practices](https://docs.databricks.com/delta/best-practices.html)

---

## Czyszczenie zasobów

Posprzątaj zasoby utworzone podczas notebooka:

In [0]:
# UWAGA: Uruchom tylko jeśli chcesz usunąć wszystkie utworzone obiekty

# Usuń tabele Bronze
# spark.sql(f"DROP TABLE IF EXISTS {CUSTOMERS_TABLE}")
# spark.sql(f"DROP TABLE IF EXISTS {ORDERS_TABLE}")

# Usuń widoki i tabele Silver
# spark.sql(f"DROP VIEW IF EXISTS {VIEW_NAME}")
# spark.sql(f"DROP TABLE IF EXISTS {MATERIALIZED_TABLE}")

# Usuń tabele Gold
# spark.sql(f"DROP TABLE IF EXISTS {GOLD_TABLE}")

# Wyczyść cache
# spark.catalog.clearCache()

print(" Zasoby zostały wyczyszczone (odkomentuj kod, aby wykonać)")